# CUB-200-2011 Bird Retrieval with Description Embedding

這個 notebook 結合 **圖片 embedding** 和 **文字描述 embedding** 來建立更精準的鳥類圖片檢索系統。

## 可用資料來源：
| 資料來源 | 類型 | 用途 |
|---------|------|------|
| `cvpr2016_cub/text_c10/` | 每張圖 10 條自然語言描述 | ⭐ 主要文字 embedding |
| `attributes/attributes.txt` | 312 種視覺屬性 | 輔助屬性描述 |
| `classes.txt` | 200 種鳥類名稱 | 類別資訊 |

In [1]:
# 安裝必要套件
!pip install faiss-cpu git+https://github.com/openai/CLIP.git
!pip install kaggle tqdm

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8mt4kqov
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8mt4kqov
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=ca804e615c9642c2dfcd4aab236bd29f704ba3c615a335908c7855cd9d331f35
  Stored in directory: /tmp/pip-ephem-wheel-cache-z1va3drc/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
# (Colab 用) 上傳 kaggle.json 並下載資料集
import os
from google.colab import files

# 上傳 kaggle.json
uploaded = files.upload()

# 建立 kaggle 資料夾
os.makedirs("/root/.kaggle", exist_ok=True)
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)

# 下載 CUB 200 2011
!kaggle datasets download -d wenewone/cub2002011

# 解壓
!unzip -q cub2002011.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/wenewone/cub2002011
License(s): CC0-1.0
 93% 1.39G/1.49G [00:01<00:00, 1.09GB/s]
100% 1.49G/1.49G [00:01<00:00, 1.30GB/s]


In [ ]:
!pip install open_clip_torch


In [ ]:
import torch
import open_clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-bigG-14',
    pretrained='laion2b_s39b_b160k'
)
model = model.to(device)
model.eval()

tokenizer = open_clip.get_tokenizer('ViT-bigG-14')


print(f"使用裝置: {device}")

## 1. 載入資料集資訊
- 類別名稱 (classes.txt)
- 屬性定義 (attributes.txt)
- 圖片描述 (text_c10/)

In [ ]:
# 路徑設定 (Colab 環境)
BASE_PATH = "/content"  # 本地請改成你的路徑
IMAGE_FOLDER = os.path.join(BASE_PATH, "CUB_200_2011/images")
TEXT_FOLDER = os.path.join(BASE_PATH, "cvpr2016_cub/text_c10")
CLASSES_FILE = os.path.join(BASE_PATH, "CUB_200_2011/classes.txt")
ATTRIBUTES_FILE = os.path.join(BASE_PATH, "CUB_200_2011/attributes/attributes.txt")

print(f"Image folder: {IMAGE_FOLDER}")
print(f"Text folder: {TEXT_FOLDER}")

In [ ]:
# 載入類別名稱
def load_classes(path):
    """讀取 200 種鳥類名稱"""
    classes = {}
    with open(path, "r") as f:
        for line in f:
            idx, name = line.strip().split(" ", 1)
            # 格式: "001.Black_footed_Albatross" -> "Black footed Albatross"
            class_name = name.split(".")[-1].replace("_", " ")
            classes[int(idx)] = class_name
    return classes

# 載入屬性定義
def load_attributes(path):
    """讀取 312 種視覺屬性"""
    attributes = {}
    with open(path, "r") as f:
        for line in f:
            idx, name = line.strip().split(" ", 1)
            # 格式: "has_bill_shape::curved" -> "curved bill shape"
            parts = name.split("::")
            if len(parts) == 2:
                attr_type = parts[0].replace("has_", "").replace("_", " ")
                attr_value = parts[1].replace("_", " ")
                attributes[int(idx)] = f"{attr_value} {attr_type}"
            else:
                attributes[int(idx)] = name.replace("_", " ")
    return attributes

# 載入
classes = load_classes(CLASSES_FILE)
attributes = load_attributes(ATTRIBUTES_FILE)

print(f"類別數量: {len(classes)}")
print(f"屬性數量: {len(attributes)}")
print(f"\n範例類別: {list(classes.items())[:5]}")
print(f"\n範例屬性: {list(attributes.items())[:10]}")

In [ ]:
# 載入每張圖片的文字描述 (text_c10)
def load_descriptions(text_folder):
    """從 text_c10 讀取每張圖片的 10 條文字描述"""
    descriptions = {}

    for class_dir in os.listdir(text_folder):
        class_path = os.path.join(text_folder, class_dir)
        if not os.path.isdir(class_path):
            continue

        for f in os.listdir(class_path):
            if f.endswith(".txt"):
                txt_path = os.path.join(class_path, f)
                # 圖片名: Black_Footed_Albatross_0001_796111.txt -> Black_Footed_Albatross_0001_796111.jpg
                img_name = f.replace(".txt", ".jpg")
                img_key = os.path.join(class_dir, img_name)

                with open(txt_path, "r", encoding="utf-8") as file:
                    descs = [line.strip() for line in file.readlines() if line.strip()]
                    descriptions[img_key] = descs

    return descriptions

descriptions = load_descriptions(TEXT_FOLDER)
print(f"載入 {len(descriptions)} 張圖片的描述")

# 顯示範例
sample_key = list(descriptions.keys())[0]
print(f"\n=== 範例圖片: {sample_key} ===")
print(f"描述數量: {len(descriptions[sample_key])}")
for i, desc in enumerate(descriptions[sample_key][:5]):
    print(f"  {i+1}. {desc}")

## 2. 產生 Image + Text Combined Embedding

結合策略：
- **Image Embedding**: CLIP encode_image
- **Text Embedding**: 對 10 條描述取平均 embedding
- **Combined Embedding**: α × Image + (1-α) × Text

In [ ]:
from tqdm import tqdm
import numpy as np
# 產生所有圖片的 embedding

combined_embeddings = []
image_embeddings = []
text_embeddings = []
paths = []
descriptions_list = []
class_names_list = []

# 權重設定
ALPHA = 0.6  # image 權重，text 權重為 1-ALPHA

print(f"開始產生 embedding (α={ALPHA})...")
print(f"Combined = {ALPHA} × Image + {1-ALPHA} × Text\n")

# 遍歷所有圖片
all_dirs = [d for d in os.listdir(IMAGE_FOLDER) if os.path.isdir(os.path.join(IMAGE_FOLDER, d))]

for class_dir in tqdm(all_dirs, desc="Processing classes"):
    class_path = os.path.join(IMAGE_FOLDER, class_dir)
    class_name = class_dir.split(".")[-1].replace("_", " ")

    for f in os.listdir(class_path):
        if not f.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img_path = os.path.join(class_path, f)
        rel_path = os.path.join(class_dir, f)

        try:
            # 1. Image Embedding
            image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
            with torch.no_grad():
                img_emb = model.encode_image(image)
                img_emb = img_emb / img_emb.norm(dim=-1, keepdim=True)

            # 2. Text Embedding (從 text_c10 描述)
            if rel_path in descriptions and descriptions[rel_path]:
                # 對所有描述取平均 embedding
                all_descs = descriptions[rel_path]
                text_tokens = tokenizer(all_descs).to(device)
                with torch.no_grad():
                    text_emb = model.encode_text(text_tokens)
                    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)
                    text_emb = text_emb.mean(dim=0, keepdim=True)  # 平均所有描述
                    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)
                desc_text = all_descs[0]  # 保存第一條描述
            else:
                # Fallback: 使用類別名稱
                fallback_text = f"a photo of a {class_name}"
                text_tokens = tokenizer([fallback_text]).to(device)
                with torch.no_grad():
                    text_emb = model.encode_text(text_tokens)
                    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)
                desc_text = fallback_text

            # 3. Combined Embedding
            combined = ALPHA * img_emb + (1 - ALPHA) * text_emb
            combined = combined / combined.norm(dim=-1, keepdim=True)

            # 儲存結果
            combined_embeddings.append(combined.cpu().numpy())
            image_embeddings.append(img_emb.cpu().numpy())
            text_embeddings.append(text_emb.cpu().numpy())
            paths.append(img_path)
            descriptions_list.append(desc_text)
            class_names_list.append(class_name)

        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue

print(f"\n✅ 完成！共處理 {len(paths)} 張圖片")

## 3. 建立 FAISS Index

In [ ]:
import faiss

# 轉換為 numpy array
combined_array = np.vstack(combined_embeddings).astype("float32")
image_array = np.vstack(image_embeddings).astype("float32")
text_array = np.vstack(text_embeddings).astype("float32")

# 建立三種 Index
# 1. Combined Index (image + text)
index_combined = faiss.IndexFlatIP(combined_array.shape[1])
index_combined.add(combined_array)

# 2. Image-only Index
index_image = faiss.IndexFlatIP(image_array.shape[1])
index_image.add(image_array)

# 3. Text-only Index
index_text = faiss.IndexFlatIP(text_array.shape[1])
index_text.add(text_array)

print("✅ FAISS Index 建立完成！")
print(f"   Combined Index: {index_combined.ntotal} 向量")
print(f"   Image Index:    {index_image.ntotal} 向量")
print(f"   Text Index:     {index_text.ntotal} 向量")

## 4. 搜尋函數

In [ ]:
import json
from IPython.display import display

# 載入或建立 path.json
def load_path_mapping(path_json_file):
    """載入 path.json 檔案建立索引到路徑的對應"""
    with open(path_json_file, 'r', encoding='utf-8') as f:
        path_data = json.load(f)
    return path_data

def create_path_json(path_json_file, paths_list):
    """根據目前的 paths 列表建立 path.json"""
    with open(path_json_file, 'w', encoding='utf-8') as f:
        json.dump(paths_list, f, ensure_ascii=False, indent=2)
    print(f"✅ 已建立 {path_json_file}，共 {len(paths_list)} 筆路徑")

# 設定 path.json 路徑
PATH_JSON_FILE = os.path.join(BASE_PATH, "path.json")

# 載入或建立路徑對應
if os.path.exists(PATH_JSON_FILE):
    path_mapping = load_path_mapping(PATH_JSON_FILE)
    print(f"✅ 已載入 path.json，共 {len(path_mapping)} 筆路徑對應")
else:
    # 如果不存在，則根據目前的 paths 列表建立
    print(f"⚠️ 找不到 {PATH_JSON_FILE}，正在建立...")
    create_path_json(PATH_JSON_FILE, paths)
    path_mapping = paths  # 直接使用 paths 列表

def get_image_path(index):
    """根據索引取得圖片路徑，優先使用 path.json"""
    if path_mapping is not None:
        # 根據 path.json 的結構取得路徑
        # 如果是 list 格式
        if isinstance(path_mapping, list):
            return path_mapping[index]
        # 如果是 dict 格式 (key 是索引的字串)
        elif isinstance(path_mapping, dict):
            return path_mapping.get(str(index), paths[index])
    # fallback 到原本的 paths 列表
    return paths[index]

def search_by_text(query, k=5, index_type="combined"):
    """
    用文字搜尋圖片

    Args:
        query: 搜尋文字
        k: 返回結果數量
        index_type: "combined", "image", "text"
    """
    text = tokenizer([query]).to(device)
    with torch.no_grad():
        query_emb = model.encode_text(text)
        query_emb = query_emb / query_emb.norm(dim=-1, keepdim=True)
        query_emb = query_emb.cpu().numpy().astype("float32")

    # 選擇 index
    if index_type == "combined":
        index = index_combined
    elif index_type == "image":
        index = index_image
    else:
        index = index_text

    distances, indices = index.search(query_emb, k)

    results = []
    for j, i in enumerate(indices[0]):
        results.append({
            "path": get_image_path(i),
            "description": descriptions_list[i],
            "class_name": class_names_list[i],
            "score": distances[0][j]
        })
    return results


def search_by_image(img_path, k=5, index_type="combined"):
    """
    用圖片搜尋相似圖片

    Args:
        img_path: 圖片路徑
        k: 返回結果數量
        index_type: "combined", "image", "text"
    """
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        query_emb = model.encode_image(image)
        query_emb = query_emb / query_emb.norm(dim=-1, keepdim=True)
        query_emb = query_emb.cpu().numpy().astype("float32")

    # 選擇 index
    if index_type == "combined":
        index = index_combined
    elif index_type == "image":
        index = index_image
    else:
        index = index_text

    distances, indices = index.search(query_emb, k)

    results = []
    for j, i in enumerate(indices[0]):
        results.append({
            "path": get_image_path(i),
            "description": descriptions_list[i],
            "class_name": class_names_list[i],
            "score": distances[0][j]
        })
    return results


def show_results(results, show_description=True):
    """顯示搜尋結果"""
    for i, r in enumerate(results):
        print(f"\n{'='*50}")
        print(f"[{i+1}] {r['class_name']} (Score: {r['score']:.4f})")
        if show_description:
            print(f"    📝 {r['description'][:100]}...")
        display(Image.open(r['path']))

## 5. 測試搜尋

In [ ]:
# 文字搜尋測試 - 使用 Combined Index
print("🔍 搜尋: 'gray kingbird' (Combined Index)")
results = search_by_text("short tail crow", k=1, index_type="combined")
show_results(results)

In [ ]:
# 文字搜尋測試 - 使用詳細描述
print("🔍 搜尋: 'a small bird with yellow belly and black wings' (Combined Index)")
results = search_by_text("a small bird with yellow belly and black wings", k=1, index_type="combined")
show_results(results)

In [ ]:
# 比較不同 Index 的搜尋結果
query = "red bird with black head"

print(f"🔍 搜尋: '{query}'\n")

print("=" * 60)
print("📊 Combined Index 結果:")
results_combined = search_by_text(query, k=3, index_type="combined")
for r in results_combined:
    print(f"  - {r['class_name']} (score: {r['score']:.4f})")

print("\n" + "=" * 60)
print("🖼️ Image-only Index 結果:")
results_image = search_by_text(query, k=3, index_type="image")
for r in results_image:
    print(f"  - {r['class_name']} (score: {r['score']:.4f})")

print("\n" + "=" * 60)
print("📝 Text-only Index 結果:")
results_text = search_by_text(query, k=3, index_type="text")
for r in results_text:
    print(f"  - {r['class_name']} (score: {r['score']:.4f})")

In [ ]:
# 圖片搜尋測試
sample_img = paths[100]  # 任選一張
print(f"🖼️ 查詢圖片: {class_names_list[100]}")
display(Image.open(sample_img))

print("\n🔍 相似圖片 (Combined Index):")
results = search_by_image(sample_img, k=1, index_type="combined")
show_results(results)

## 6. 儲存 Index 和資料

In [ ]:
import pickle

# 儲存 FAISS Index
faiss.write_index(index_combined, "cub200_combined.index")
faiss.write_index(index_image, "cub200_image.index")
faiss.write_index(index_text, "cub200_text.index")

# 儲存 metadata
metadata = {
    "paths": paths,
    "descriptions": descriptions_list,
    "class_names": class_names_list,
    "alpha": ALPHA
}

with open("cub200_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ 已儲存:")
print("   - cub200_combined.index")
print("   - cub200_image.index")
print("   - cub200_text.index")
print("   - cub200_metadata.pkl")

In [ ]:
# 載入已儲存的 Index (之後使用)
import pickle
import faiss
#
# # 載入 Index
# index_combined = faiss.read_index("cub200_combined.index")
# index_image = faiss.read_index("cub200_image.index")
# index_text = faiss.read_index("cub200_text.index")
#
# # 載入 metadata
# with open("cub200_metadata.pkl", "rb") as f:
#     metadata = pickle.load(f)
#     paths = metadata["paths"]
#     descriptions_list = metadata["descriptions"]
#     class_names_list = metadata["class_names"]
#
# print(f"載入完成！共 {index_combined.ntotal} 向量")